In [19]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from datetime import datetime

In [20]:
# API key
from config import weather_api_key

In [21]:
# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [22]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=25)
lngs = np.random.uniform(low=-180.000, high=180.000, size=25)
lat_lngs = zip(lats, lngs)
lat_lngs

In [23]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [24]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

23

In [25]:
# Empty list to hold the weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes    
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # Create endpoint URL with each city    
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city    
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count    
    record_count += 1
    
# Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data        
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data        
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # Convert the date to ISO standard        
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into city_data list        
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | karaton
Processing Record 2 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 3 of Set 1 | mar del plata
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | cidreira
Processing Record 6 of Set 1 | new norfolk
Processing Record 7 of Set 1 | atuona
Processing Record 8 of Set 1 | kyaukse
Processing Record 9 of Set 1 | butaritari
Processing Record 10 of Set 1 | miyako
Processing Record 11 of Set 1 | punta arenas
Processing Record 12 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 13 of Set 1 | port alfred
Processing Record 14 of Set 1 | hobart
Processing Record 15 of Set 1 | kunming
Processing Record 16 of Set 1 | lyubeshiv
City not found. Skipping...
Processing Record 17 of Set 1 | chik
Processing Record 18 of Set 1 | albany
Processing Record 19 of Set 1 | ancud
Processing Record 20 of Set 1 | hit
Processing Record 21 of Set 1 | rikitea
Proces

In [26]:
# Convert the array of dictionaries to a Pandas DataFrame

city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Karaton,-6.3424,106.1045,87.96,65,49,3.22,ID,2021-05-28 01:42:16
1,Mar Del Plata,-38.0023,-57.5575,64.00,48,0,10.00,AR,2021-05-28 01:42:16
2,Ushuaia,-54.8000,-68.3000,28.06,100,75,17.27,AR,2021-05-28 01:41:39
3,Cidreira,-30.1811,-50.2056,60.93,76,0,9.60,BR,2021-05-28 01:42:16
4,New Norfolk,-42.7826,147.0587,49.75,40,67,7.11,AU,2021-05-28 01:42:17
5,Atuona,-9.8000,-139.0333,78.80,74,56,13.91,PF,2021-05-28 01:42:17
6,Kyaukse,21.6000,96.1333,89.60,62,20,7.74,MM,2021-05-28 01:42:17
7,Butaritari,3.0707,172.7902,83.91,78,64,16.20,KI,2021-05-28 01:42:18
8,Miyako,39.6367,141.9525,64.44,71,79,2.51,JP,2021-05-28 01:38:08
9,Punta Arenas,-53.1500,-70.9167,32.02,86,0,14.67,CL,2021-05-28 01:42:13


In [30]:
new_column_order = ["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]
city_data_new_df = city_data_df[new_column_order]
city_data_new_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Karaton,ID,2021-05-28 01:42:16,-6.3424,106.1045,87.96,65,49,3.22
1,Mar Del Plata,AR,2021-05-28 01:42:16,-38.0023,-57.5575,64.00,48,0,10.00
2,Ushuaia,AR,2021-05-28 01:41:39,-54.8000,-68.3000,28.06,100,75,17.27
3,Cidreira,BR,2021-05-28 01:42:16,-30.1811,-50.2056,60.93,76,0,9.60
4,New Norfolk,AU,2021-05-28 01:42:17,-42.7826,147.0587,49.75,40,67,7.11


In [33]:
# Create the output file (CSV)
output_data_file = "weather_data/cities.csv"

# Export the City_Data into a CSV.
city_data_new_df.to_csv(output_data_file, index_label="City_ID")